In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import math
import multiprocessing as mp

In [3]:
days_left = 365*1
# days_left = 10
today='2021-09-02'

In [4]:
def get_securities(date):
    page_size = 100
    all_rows = []
    start = 0
    url_temp = "https://iss.moex.com/iss/history/engines/stock/markets/bonds/securities.json?date={date}&start={start}&iss.json=extended"
    r = requests.get(url_temp.format(start=start, date=date))
    res = r.json()
    all_rows += res[1]['history'][1]
    total = res[1]['history.cursor'][1][0]['TOTAL']
    pages = math.ceil(total/page_size)+1
    print("Date:{} Total:{}, Pages:{}".format(date, total, pages))
    if total <=100:
        return all_rows
    for page in range(1, pages):
        start = page*page_size
        r = requests.get(url_temp.format(start=start, date=date))
        res = r.json()
        all_rows += res[1]['history'][1]
    return all_rows

# res = get_securities("2020-09-01")



In [80]:
secs = []
td = datetime.today()- timedelta(1)
dates = []
for i in range(days_left):
    dt = td - timedelta(i)
    dates.append(dt.strftime('%Y-%m-%d'))
    

In [ ]:
# pool = mp.Pool(mp.cpu_count())
# results = pool.map(get_securities, [date for date in dates])
# pool.close()

Date:2021-09-03 Total:2376, Pages:25
Date:2021-07-19 Total:2344, Pages:25
Date:2021-09-02 Total:2378, Pages:25
Date:2021-07-18 Total:0, Pages:1
Date:2021-07-17 Total:0, Pages:1
Date:2021-07-16 Total:2341, Pages:25
Date:2021-09-01 Total:2379, Pages:25
Date:2021-08-31 Total:2377, Pages:25
Date:2021-07-15 Total:2343, Pages:25
Date:2021-07-14 Total:2349, Pages:25
Date:2021-08-30 Total:2377, Pages:25
Date:2021-08-29 Total:0, Pages:1
Date:2021-08-28 Total:0, Pages:1
Date:2021-08-27 Total:2375, Pages:25
Date:2021-08-26 Total:2373, Pages:25
Date:2021-08-25 Total:2378, Pages:25
Date:2021-07-13 Total:2341, Pages:25
Date:2021-08-24 Total:2377, Pages:25
Date:2021-07-12 Total:2336, Pages:25
Date:2021-08-23 Total:2375, Pages:25
Date:2021-07-11 Total:0, Pages:1
Date:2021-07-10 Total:0, Pages:1
Date:2021-07-09 Total:2336, Pages:25
Date:2021-08-22 Total:0, Pages:1
Date:2021-08-21 Total:0, Pages:1
Date:2021-08-20 Total:2372, Pages:25
Date:2021-07-08 Total:2337, Pages:25
Date:2021-08-19 Total:2376, Pages

Date:2021-02-09 Total:2270, Pages:24
Date:2021-02-08 Total:2263, Pages:24
Date:2020-12-24 Total:2243, Pages:24
Date:2021-02-07 Total:0, Pages:1
Date:2021-02-06 Total:0, Pages:1
Date:2021-02-05 Total:2255, Pages:24
Date:2020-12-23 Total:2241, Pages:24
Date:2020-12-22 Total:2233, Pages:24
Date:2020-12-21 Total:2226, Pages:24
Date:2020-12-20 Total:0, Pages:1
Date:2020-12-19 Total:0, Pages:1
Date:2020-12-18 Total:2225, Pages:24
Date:2021-02-04 Total:2256, Pages:24
Date:2020-12-17 Total:2224, Pages:24
Date:2020-12-16 Total:2227, Pages:24
Date:2021-02-03 Total:2261, Pages:24
Date:2020-12-15 Total:2224, Pages:24
Date:2021-02-02 Total:2258, Pages:24
Date:2021-02-01 Total:2256, Pages:24
Date:2021-01-31 Total:0, Pages:1
Date:2021-01-30 Total:0, Pages:1
Date:2021-01-29 Total:2254, Pages:24
Date:2020-12-14 Total:2223, Pages:24
Date:2020-12-13 Total:0, Pages:1
Date:2020-12-12 Total:0, Pages:1
Date:2020-12-11 Total:2213, Pages:24
Date:2020-12-10 Total:2205, Pages:24
Date:2020-12-09 Total:2206, Pages

In [ ]:
secs = []
for r in results:
    secs += r

In [5]:
df = pd.DataFrame(get_securities(today))
#df.to_excel("moex_bonds.xlsx", index=False)


Date:2021-09-02 Total:2378, Pages:25


In [9]:
df[['BOARDID','TRADEDATE','SECID','WAPRICE','DURATION','YIELDATWAP', 'COUPONPERCENT','FACEUNIT','FACEVALUE']]


,BOARDID,TRADEDATE,SECID,WAPRICE,DURATION,YIELDATWAP,COUPONPERCENT,FACEUNIT,FACEVALUE
0,TQOD,2021-09-02,AMUNIBB2AER2,103.20,NaN,NaN,5.25,USD,100.0
1,TQOD,2021-09-02,AMUNIBB2DER6,103.60,835.0,3.78,5.30,USD,100.0
2,TQCB,2021-09-02,RU000A0JNYN1,99.87,277.0,6.25,6.00,RUB,1000.0
3,TQIR,2021-09-02,RU000A0JQ5C5,NaN,NaN,NaN,5.00,RUB,1000.0
4,TQCB,2021-09-02,RU000A0JQ7Z2,101.97,943.0,7.19,7.85,RUB,1000.0
...,...,...,...,...,...,...,...,...,...
2373,TQOD,2021-09-02,XS2369925511,100.00,NaN,NaN,0.00,USD,1250.0
2374,TQOD,2021-09-02,XS2371857223,NaN,NaN,NaN,0.00,USD,1250.0
2375,TQOD,2021-09-02,XS2376055864,NaN,NaN,NaN,0.00,USD,1250.0
2376,TQOD,2021-09-02,XS2377117697,100.00,NaN,NaN,0.00,USD,1250.0


In [10]:
source_file = "data.xlsx" # исходник данных для получения universe
main_data = pd.read_excel(source_file, index_col=None, engine="openpyxl", parse_dates=True, sheet_name = "universe")
bonds = main_data[main_data['asset_class'] == 'Облигации высокодоходные']

,asset_class,country,currency,asset,source,code,isin,issuer,sector,logo,hash,rating,credit_risk_score,duration,maturity,ytm,gspread
0,Акции РФ,RU,RUB,Газпром,finance.yahoo.com,GAZP.ME,NaN,Газпром,Energy Minerals,https://finnhub.io/api/logo?symbol=GAZP.ME,7fa93e8781319d63f347c23b5771392a,BBB-,5,NaN,NaN,NaN,NaN
1,Акции РФ,RU,RUB,ГМК Норильский никель,finance.yahoo.com,GMKN.ME,NaN,ГМК Норильский никель,Metals & Mining,https://finnhub.io/api/logo?symbol=GMKN.ME,43abbcafd564f2ecc6ca2fb187d1fb8e,BBB-,5,NaN,NaN,NaN,NaN
2,Акции РФ,RU,RUB,Лукойл,finance.yahoo.com,LKOH.ME,NaN,Лукойл,Energy Minerals,https://finnhub.io/api/logo?symbol=LKOH.ME,4172b50533890d6511949e87f1190212,BBB-,5,NaN,NaN,NaN,NaN
3,Акции РФ,RU,RUB,Магнит,finance.yahoo.com,MGNT.ME,NaN,Магнит,Retail,https://finnhub.io/api/logo?symbol=MGNT.ME,16f7ad884bb6bf80fed588974a6f09fa,BB+,4,NaN,NaN,NaN,NaN
4,Акции РФ,RU,RUB,МТС,finance.yahoo.com,MTSS.ME,NaN,МТС,Telecommunication,https://finnhub.io/api/logo?symbol=MTSS.ME,901adbf9df2318896f39a1629cbd37b3,BBB-,5,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,Облигации высокодоходные,RU,RUB,"Коммерческая недвижимость ФПК Гарант-Инвест, 0...",navigator.trackrecords.ru,RU000A1005T9,RU000A1005T9,Коммерческая недвижимость ФПК Гарант-Инвест,NaN,NaN,NaN,B,0,2.591781,05.04.2024,12.541504,569.597213
88,Облигации высокодоходные,RU,RUB,"Джи-групп, 002Р-01",navigator.trackrecords.ru,RU000A103JR3,RU000A103JR3,Джи-групп,NaN,NaN,NaN,B+,1,2.942466,12.08.2024,10.570679,369.848090
89,Облигации высокодоходные,RU,RUB,"ГК Пионер, 001P-05",navigator.trackrecords.ru,RU000A102KG6,RU000A102KG6,ГК Пионер,NaN,NaN,NaN,B+,1,3.290411,17.12.2024,10.227051,334.862530
90,Облигации высокодоходные,RU,RUB,"ТрансФин-М, 001Р-07",navigator.trackrecords.ru,RU000A103M36,RU000A103M36,ТрансФин-М,NaN,NaN,NaN,B+,1,2.980822,26.08.2024,10.139160,326.562755
